<a href="https://colab.research.google.com/github/Navifra-Denny/01_linear_algegra/blob/main/01.pca/principal_component_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [2]:
SLOPE_MIN = -50.0
SLOPE_MAX = 50.0

# ===== 1. 데이터 생성 함수 =====
def generate_clipped_gaussian(mean, std, min_val, max_val, size):
    data = np.random.normal(mean, std, size)
    data = np.clip(data, min_val, max_val)
    return np.round(data, 1)

In [19]:
# ===== 2. PCA 주성분 방향 계산 =====
def calculate_pca_vectors(x, y):
    data = np.column_stack((x, y))
    mean = np.mean(data, axis=0)
    centered = data - mean
    cov = np.cov(centered, rowvar=False)
    eigvals, eigvecs = np.linalg.eigh(cov)  # ascending order
    pc1 = eigvecs[:, 1]  # largest eigenvalue
    pc2 = eigvecs[:, 0]  # smallest eigenvalue (orthogonal)
    return pc1, pc2, mean

## SS (Sumo of Squares)

![ss](https://raw.githubusercontent.com/Navifra-Denny/01_linear_algegra/main/01.pca/img.png)


> 모든 데이터에서 원점(평균으로 이동된 점)을 지나는 직선에 수선을 발을 내리면, 원점으로부터 수선의 발까지의 길이를 구할 수 있게 된다. 원점을 지나는 직선의 기울기가 변함에 따라, 이 빨간선들의 길이 또한 변하게 된다. PCA 에서는 이 빨간선들의 길이 제곱들의 합이 최대가 되는 직선을 찾는다. 이것이 SS(Sum of Squares) 이다.


## 요약
* 직선은 평균점을 지나며, 기울기 𝑚 에 따라 정의됨:
$$ y = m(x - \bar{x}) + \bar{y} $$
* 각 데이터 포인트 $(x_i, y_i)$ 에 대해 직선에 내린 **수선의 발**을 구하고,
* 그 **수선 길이의 제곱**을 계산한 후, 전체에 대한 합산:
$$ SS(m) = \sum_{i=1}^{N} d_i^2 $$


### 직선과 점 사이의 거리 계산 공식
* 직선: $y=mx+b$
* 점: $(x_0, y_0)$
* 직선과 점 사이 거리:
$$ d = \frac{\left| mx_0 - y_0 + b \right|}{\sqrt{m^2 + 1}} $$


In [20]:
# ===== 3. SS 계산 함수 (벡터 투영 기반) =====
def calculate_ss_from_vector(x, y, direction_vector):
    data = np.column_stack((x, y))
    mean = np.mean(data, axis=0)
    centered = data - mean

    # 수직 방향 벡터 구하기
    perp_vector = np.array([-direction_vector[1], direction_vector[0]])
    perp_vector = perp_vector / np.linalg.norm(perp_vector)

    projections = centered @ perp_vector
    ss = np.sum(projections**2)
    return ss


In [21]:
# ===== 4. 사용자 기울기를 벡터로 변환 =====
def slope_to_unit_vector(slope):
    vec = np.array([1.0, slope])
    return vec / np.linalg.norm(vec)

In [22]:
# num_samples = 300

# # Gaussian parameters
# height_mean, height_std = 170, 6.8
# weight_mean, weight_std = 80, 13

# # Generate height and weight with clipping and rounding
# heights = generate_clipped_gaussian(height_mean, height_std, 0, 210, num_samples)
# weights = generate_clipped_gaussian(weight_mean, weight_std, 40, 150, num_samples)

# # Stack into (N, 2) dataset
# dataset = np.column_stack((heights, weights))

# # Compute means
# height_avg = np.mean(heights)
# weight_avg = np.mean(weights)


In [27]:
# ===== 5. 데이터 생성 및 PCA 벡터 계산 =====
num_samples = 300
heights = generate_clipped_gaussian(170, 17, 0, 300, num_samples)
weights = generate_clipped_gaussian(80, 13, 40, 150, num_samples)

pca_vec1, pca_vec2, center = calculate_pca_vectors(heights, weights)

In [32]:
# ===== 6. 시각화 업데이트 함수 =====
def update_plot(m):
    user_vec = slope_to_unit_vector(m)
    ss_user = calculate_ss_from_vector(heights, weights, user_vec)
    ss_pca = calculate_ss_from_vector(heights, weights, pca_vec1)
    ss_ortho = calculate_ss_from_vector(heights, weights, pca_vec2)

    plt.figure(figsize=(8, 6))
    plt.scatter(heights, weights, color='skyblue', edgecolors='black', label='Data')

    # 평균점
    plt.axvline(x=center[0], color='red', linestyle='--', linewidth=0.5)
    plt.axhline(y=center[1], color='red', linestyle='--', linewidth=0.5)

    # 사용자 직선
    x_vals = np.linspace(center[0] - 50, center[0] + 50, 100)
    y_user = m * (x_vals - center[0]) + center[1]
    plt.plot(x_vals, y_user, color='green', label=f'Slope={m:.2f} | SS={ss_user:.1f}', linewidth=2)

    # PCA 1 (분산 최대 방향)
    slope_pca = pca_vec1[1] / pca_vec1[0]
    y_pca = slope_pca * (x_vals - center[0]) + center[1]
    plt.plot(x_vals, y_pca, color='orange', linestyle='--', linewidth=2,
             label=f'PCA 1 (SS={ss_pca:.1f})')

    # PCA 2 (직교 방향)
    slope_ortho = pca_vec2[1] / pca_vec2[0]
    y_ortho = slope_ortho * (x_vals - center[0]) + center[1]
    plt.plot(x_vals, y_ortho, color='purple', linestyle='--', linewidth=2,
             label=f'Ortho (SS={ss_ortho:.1f})')

    # 보기 편한 축 범위
    plt.xlim(center[0] - 40, center[0] + 40)
    plt.ylim(center[1] - 40, center[1] + 40)
    plt.title("PCA vs Perpendicular Distance (SS)")
    plt.xlabel("Height (cm)")
    plt.ylabel("Weight (kg)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [33]:
# 슬라이더: 기울기 m (-5 ~ 5 사이 실수)
slope_slider = widgets.FloatSlider(
    value=0.0,
    min=SLOPE_MIN,
    max=SLOPE_MAX,
    step=0.1,
    description='Slope (m):',
    continuous_update=True
)

widgets.interact(update_plot, m=slope_slider);

interactive(children=(FloatSlider(value=0.0, description='Slope (m):', max=50.0, min=-50.0), Output()), _dom_c…